# Notebook to calculate dice distances between organisms

This notebook creates a neighbour joining tree of organisms by comparing their protein content using blastp, and scoring strength organism-organism relationship based on a ratio of the sum of their total bitscores from the blastp results. This ratio is called the a dice distance.

Things necessary for this script to work:  
* all proteins in an AA fasta file labelled as "genome|prot"
* An all-vs-all blastp results
```bash
blastp -db cat_all_viruses2blast.faa -query cat_all_viruses2blast.faa -out VCproteins_refseq_selfblastp_hsp1.out -outfmt 6 -num_threads 20 -max_hsps 1
```

Import libraries

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import csv, sys, re

Read in blast results

In [ ]:
#read the blast file (all_prot_selfdmnd.out). this file is the self blast of all the protein sequences incl. reference seq

#blastp=sys.argv[1]
selfblast='VCproteins_refseq_selfblastp_hsp1.out'

#intermediate filtered blast file
temp1='VCproteins_refseq_selfblastp_hsp1_filtered.out'

#outfile is pairwise alignment file
outfile='VCprot_refseqprot_pw.tsv'

In [ ]:
#read blast file into a dataframe 'df' and rename columns, splits on tab and pipe
print("reading in blast results...")
df_self=pd.read_csv(selfblast,sep='\t|\|',usecols=[0,1,2,3,4,12,13],header=None,names=['qcontig','qprot','scontig','sprot','pident','evalue','bitscore'],engine='python')
print(df_self)

filter blast results: evalue <= 0.00001 (1E-5) and pident >= 40% should I use a different filter here?

In [ ]:
print("filtering blast results...")
df_self_1=df_self.loc[(df_self['evalue'] <= 0.00001) & (df_self['pident'] >= 40)]
#print(df_self_1)
df_self_1.to_csv(temp1,sep='\t',header=True,index=False)

#check if any there are any NAs
df1=df_self_1[df_self_1.isna().any(axis=1)]
#print(df1)

sum bitscores by query and subject contigs and put into condensed datafram 'df_cond'

In [ ]:
print("calculating bitscore sums...")
df_cond=df_self_1.groupby(['qcontig','scontig']).sum().reset_index()
df_cond.rename(columns={'bitscore':'BSsum'},inplace=True)
df_cond=df_cond[['qcontig','scontig','BSsum']]
print("finished calculating bitscore sums...")
#print(df_cond)

mapping contigs to dictionary

In [ ]:
contigs=[]
for line in df_cond['qcontig']:
    if line in contigs:
        next
    else:
        contigs.append(line)
#print(contigs)

dictionary={}
i=0
for j in contigs:
    dictionary[j]=i
    i=i+1
#print(dictionary)

convert dataframe from long to wide format and then into a 2d array

In [ ]:
df_cond.dropna(inplace=True)

df_cond_1=df_cond.pivot(index='qcontig',columns='scontig',values='BSsum')
print("pivoted matrix...")

#only to check that there's something in the matrix.
#df_cond.to_csv("./temp1.tsv",sep='\t',header='True')

#print(df_cond)
print(df_cond_1)

arr=df_cond_1.values
#print(len(arr)-1)

calculate dice distance and assign to new dictionary 'new_dict'

In [ ]:
print("calculating dice distances...")
new_dict={}
for i in range(0,len(arr)):
    for j in range(0,len(arr[0])):
        dice=1-((2*max(arr[i][j],arr[j][i]))/(arr[i][i]+arr[j][j]))
        new_dict[i,j]=dice

create a new final dataframe for printing using list comprehension and using values from inv_map (inversed contig mapping) and dice from 'new_dict'

In [ ]:
#inverse dictionary of contigs
print("inverse mapping all distance results to dataframe")
inv_map={v:k for k,v in dictionary.items()}
#
#create a new final dataframe for printing using list comprehension and using values from inv_map (inversed contig mapping) and dice from 'new_dict'
d=[]
for k1,k2 in new_dict.keys():
    d.append({'contig1':inv_map[k1],'contig2':inv_map[k2],'dice':new_dict[k1,k2]})
#    print(inv_map[k1]+'\t'+inv_map[k2]+'\t'+str(new_dict[k1,k2]))
#print(d)

create final dataframe for printing out and clean up

In [ ]:
print("final clean up...")
df_final=pd.DataFrame(d)
df_final=df_final[['contig1','contig2','dice']]
df_final.replace([np.inf,-np.inf],np.nan,inplace=True)
df_final.dropna(inplace=True)
#df_final.insert(3,'1-dice',1-df_final['dice'])
#df_final.drop(columns='dice',inplace=True)

print("printing results to file...")
df_final.to_csv(outfile,sep='\t',header=True,index=False)
#print(df_final)

print("done! time for a beer...")